In [44]:

import spacy 
import pandas as pd
import numpy as np
#import textract


dataAudio = pd.read_csv("../Data procesada/data_audio.csv")


def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i] <= array[begin]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

# Entra el texto y aplica el modelo de ner de spacy

def LabelsLG(text):

#    smLoad = spacy.load("es_core_news_sm")
    lgLoad = spacy.load("es_core_news_lg")
    
    entities = []
    labels = []
    df = pd.DataFrame([], columns=["Entities", 'Labels'])
    
    trainedText = lgLoad(text)
#    trainedText = smLoad(text)

    for ent in trainedText.ents:
        entities.append(ent)
        labels.append(ent.label_)
        df = pd.concat([df,pd.DataFrame({'Entities':entities,'Labels':labels})])
            
    df["textEntities"] = df.Entities.apply(lambda x : x.text)
    
    return df

# Guarda las entidades "PER" en una serie

def GetPersonsCount (text):
    
    df = LabelsLG(text)
    
    return df.textEntities[df.Labels=="PER"].value_counts()


# Agrega nuevas columnas de personas al df que estan en la serie pero no en el df

def AddPerson (dataFrame,seriePerson):
    
    for person in seriePerson.index:
        if not person in dataFrame.columns:
            dataFrame[person] = 0
            
    return dataFrame

# Agrega una fila vacia al df

def AddEmptyRow (dataFrame):
    
    newRow = pd.DataFrame([[0] * dataFrame.shape[1]], columns=dataFrame.columns)
    dataFrame = pd.concat([dataFrame, newRow], ignore_index=True)
    
    return dataFrame
 
# Se agregan al df la informacion producida por el modelo de ner    
    
def AddContent (dataFrame, text, channel): #toma un dataframe y un texto

    personCount =  GetPersonsCount (text)

    dataFrame = AddPerson (dataFrame,personCount)

    dataFrame = AddEmptyRow(dataFrame)

    dataFrame.loc[dataFrame.index[-1],"Text"] = text
    dataFrame.loc[dataFrame.index[-1],"Channel"] = channel

    for person in personCount.index:
        dataFrame.iloc[-1, dataFrame.columns.get_loc(person)] = personCount[person]

    return dataFrame

def DataFrameGenerator (DataFrame, textChannelSeries): #toma un df y una serie de textos
        
    for row in textChannelSeries.index:
        DataFrame = AddContent(DataFrame,textChannelSeries.textDelta[row], textChannelSeries.fuente[row])
        print(DataFrame.shape)
        
    
    return DataFrame

# Ordenador de columnas

def SortDataFrame (dataFrame):
    sums = dataFrame.sum()
    return dataFrame[sums.sort_values(ascending=False).index]
    
    

In [3]:
dataAudio.head(3)

,Unnamed: 0,folder,path_audio,file,date,text,start,end
0,0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola hola muy buenas tardes bienvenidos al dia...,2020-08-14 17:56:14,2020-08-14 17:56:33
1,1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,tenemos que hablar de esta definición por part...,2020-08-14 17:56:34,2020-08-14 17:57:19
2,2,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,algún punto empieza a reconocer también y si u...,2020-08-14 17:57:19,2020-08-14 17:58:04


In [45]:
texts = dataAudio.text
dataFrameEmpty = pd.DataFrame([], columns = ["Channel","Text"])

In [42]:
texts.shape

(317,)

In [46]:
dataFrameFinal = DataFrameGenerator(dataFrameEmpty, textsFinal)

(1, 37)
(2, 47)
(3, 64)
(4, 97)
(5, 118)
(6, 138)
(7, 164)
(8, 184)


In [47]:
dataFrameFinal

,Channel,Text,pablo,gerardo morales,alberto fernández,julián,patricia,hola hola,darío baño,pedro sánchez bueno,...,messi,miren,caminar lacan,luis suárez,diego,lebron james luzca hamilton,leonel,guardiola,piqué,abajo riquelme
0,C5N,hola hola muy buenas tardes bienvenidos al dia...,241,144,136,117,76,75,70,62,...,0,0,0,0,0,0,0,0,0,0
1,C5N,la gente circula porque los negocios se han ab...,45,0,52,0,25,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,C5N,día del niño niña o totalmente inédito como ca...,0,0,118,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,C5N,con siri poco la hipocresía y con la mentira e...,0,0,59,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TN,de los colores que se habían ido arrojó de las...,0,0,353,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,TN,pregunta relacionada con esta es posible eso s...,0,0,120,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,TN,digamos es una oportunidad desperdiciada básic...,0,0,51,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,TN,nosotros sabemos que cerca de cinco mil chicos...,0,149,0,0,0,0,0,0,...,41,34,31,30,28,22,18,14,11,4


In [50]:
dataFrameFinal.drop(columns=["Text","Channel"]).sum().sort_values(ascending = False ).head(50)

alberto fernández                     889
horacio rodríguez larreta             366
san martín                            364
gustavo sáenz                         321
macri                                 320
gerardo morales                       293
pablo                                 286
mauricio macri                        267
alberto                               245
cristóbal lópez                       195
sergio                                191
hola                                  185
nico                                  176
neil                                  159
agustín rossi                         138
fernando                              136
rodrigo fuerte                        136
marta fernández                       133
lic papelón                           130
facundo astudillo                     125
facundo                               125
hola caro                             122
julián                                117
disculpen disculpe                

mauricio pol
mauricio macri
martínez de mauricio macri


In [27]:
dataAudiofinal = pd.read_csv("../Data procesada/data_final.csv")
#dataAudiofinal

In [28]:
textsFinal = dataAudiofinal[["fuente","textDelta"]]

In [29]:
textsFinal

,fuente,textDelta
0,C5N,hola hola muy buenas tardes bienvenidos al dia...
1,C5N,la gente circula porque los negocios se han ab...
2,C5N,día del niño niña o totalmente inédito como ca...
3,C5N,con siri poco la hipocresía y con la mentira e...
4,TN,de los colores que se habían ido arrojó de las...
5,TN,pregunta relacionada con esta es posible eso s...
6,TN,digamos es una oportunidad desperdiciada básic...
7,TN,nosotros sabemos que cerca de cinco mil chicos...
